In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.io import fits
import scipy.ndimage
from astropy import units as u

from scalessim.io import *
from scalessim.focal_plane import *
from scalessim.targs import *
from scalessim.base import *
from scalessim.detectors import *
from scalessim.widgets import *
from scalessim.psfs import *
from scalessim.scales import *


%load_ext autoreload
%autoreload 2

## Choose SCALES mode and setup output directory

In [2]:
scalesmode = mode_select(start=0)
scalesmode

Dropdown(description='Mode:', options=('SED: 2.0-5.2', 'K: 1.95-2.45', 'L: 2.9-4.15', 'M: 4.5-5.2', 'H2O: 2.0-…

In [3]:
sim = SCALES(scalesmode)

In [4]:
tot_h = 1.0
nexps = 2

odir = 'cube_examples/'

## Choose guide star mag (and generate realistic PSFs)

In [5]:
guidestar = gs_select()
guidestar

Dropdown(description='GS Mag:', options=(5, 12, 13, 14), value=5)

## Set up PSFs and A star target

In [6]:
if os.path.isdir(odir)==False:
    os.mkdir(odir)
PSF = PSFs(guidestar)

In [7]:
psfs = PSF.PSF_sequence(nframes=nexps, vortex=False)

In [9]:
T_s = 10000 #stellar temperature in K
logg = 4.5 #stellar log g
zz = 0.0 #stellar metallicity
rstar = 2.0 #stellar radius
dstar = 10 #stellar distance in pc

targ, targ_bg = star_and_bkg(T_s=T_s,logg=logg,zz=zz,rstar=rstar,dstar=dstar)  

## generate cubes using scalessim target

In [17]:
Astar_ims,cube_lams = sim.cube(targ=targ,psf=psfs,verbose=True,dit=tot_h*3600/nexps)

In [11]:
print(Astar_ims.shape)

(2, 27, 108, 108)


In [12]:
pyfits.writeto(odir+'A_star_cube_0.fits',Astar_ims,overwrite=True)

## generate cubes by convolving user-made scene with PSF

In [13]:
##load oversampled wavelength spacing from scalessim
wav = sim.fp.lam.value

##load spectrum for planet
lam,plan = sonora_planet(T_p=300,sg=100,rp=1.0,d=dstar)

##downsample planet spectrum
plan2 = spectres(wav, lam, plan)

##load spectrum for star
lam,star = phoenix_star(T_s=T_s,logg=logg,zz=zz,rstar=rstar,dstar=dstar)

##downsample star spectrum
star2 = spectres(wav,lam,star)

##put star and planet into 341x108x108 scene
scene = np.zeros([len(wav),108,108])
scene[:,54,54] = star2
scene[:,44,44] = plan2


In [14]:
#convolve scene with PSFs
convscene = PSF.convolve(psfs,scene)*u.erg/u.cm/u.cm/u.s/u.um

In [18]:
Astar_ims_2,cube_lams_2 = sim.cube(cube_in=convscene,dit=tot_h*3600/nexps)

In [16]:
pyfits.writeto(odir+'A_star_cube_1.fits',Astar_ims,overwrite=True)